In [ ]:
import pandas as pd
import datetime

class SOT(object):
    def __init__(self, sot_file):
        self.sot_df = pd.read_csv(sot_file)
        self.sot_df['Date'] = pd.to_datetime(self.sot_df['Date'], dayfirst=True)
        self.sot_df['budget_date'] = pd.to_datetime(self.sot_df['budget_date'], dayfirst=True)
        
    def __call__(self):
        return self.sot_df
        vold
    def save_csv(self):
        print(datetime.datetime.now().strftime("%Y-%m-%d-%H:%M"))
    
    def update_budget(self, budget_records):
        return budget_records

    def prep_second_bank(self, csv_location):
        second_bank = pd.read_csv(csv_location)
        second_bank['Date'] = pd.to_datetime(second_bank['Date'], dayfirst=True)
        second_bank.columns = [col.strip() for col in second_bank]
        return second_bank.sort_index()
    
    def update_bank(self, csv_location):
        second_bank_df = self.prep_second_bank(csv_location)
        updated_sot = self.sot_df
        for n, row in second_bank_df.iterrows():
            updated_sot = self.bank_row_type(row, second_bank_df, updated_sot)
        return updated_sot
    
    def match_bank_on_sot_bank_conditions(self, row):
        helper = []
        for col in row.index:
            helper+= ["(updated_sot['{0}'] == row['{0}'])".format(col)]
        return'updated_sot[{}]'.format('&'.join(helper))
    
    def match_bank_on_sot_budget_conditions(self,row):
        date_match = "(abs((updated_sot['budget_date'] - row['Date'])) < pd.to_timedelta('4 Days'))"
        value_match = "(updated_sot['Expense Amount'] == row['Value'])"
        matches = [value_match, date_match]
        return 'updated_sot[{}]'.format('&'.join(matches))
    
    def bank_row_type(self, row, bank_df, updated_sot):
        # TODO need to think about rare case where updated_sot.groupby(['Description','Date', 'Value']).size().sort_values(ascending=False) 
        matched_bank_cols = eval(self.match_bank_on_sot_bank_conditions(row))
        matched_bank_number = matched_bank_cols.shape[0]
        matched_bank_index = list(matched_bank_cols.index.values)
        #print(updated_sot['budget_date']- row['Date'])
        #print(abs((updated_sot['budget_date'] - row['Date'])) < timedelta('4 Days'))
        matched_on_budget = eval(self.match_bank_on_sot_budget_conditions(row))
        matched_budget_index = list(matched_on_budget.index.values)
        print(matched_bank_index, matched_budget_index)
        already_reconciled = set(matched_bank_index) == set(matched_budget_index)

        if not matched_bank_number:
            # New bank entry...
            matched_budget = (updated_sot['budget_date'] == row['Date'])
            number_budget_matches = matched_budget.sum()
            if number_budget_matches:
                # Matches 
                if number_budget_matches == 1:
                    correct_row = updated_sot
                    for key in row.index:
                        updated_sot = updated_sot.set_value(matched_bank_index[0], updated_sot[key],row[key])
                    return updated_sot
                elif number_budget_matches > 1:
                    return updated_sot  # Do nothing
            else:
                new_row = pd.DataFrame(data=row, index=row.index).transpose()
                return pd.concat([updated_sot, new_row])  # Add new row to updated_df
        else:
            # Entry already found in bank section, no need to do anything
            return updated_sot     
    


sot = SOT('first_SOT.csv')



In [ ]:
x = sot.update_bank('25.11-25.12 -- readied.csv')


In [ ]:
x